# Sentiment Analysis of Amazon Product Reviews

### Import libraries and download data

In [2]:
import os
import json
import gzip
from urllib.request import urlopen
import numpy as np
import pandas as pd
import random
import string
import nltk
from nltk.tokenize import WhitespaceTokenizer
from nltk.corpus import stopwords
from nltk import classify
from nltk import NaiveBayesClassifier
import string
import re
import itertools

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Kindle_Store_5.json.gz

--2021-08-16 16:03:53--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Kindle_Store_5.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 538673481 (514M) [application/octet-stream]
Saving to: ‘Kindle_Store_5.json.gz’

Kindle_Store_5.json 100%[===================>] 513.72M  56.1MB/s    in 9.8s    

2021-08-16 16:04:03 (52.6 MB/s) - ‘Kindle_Store_5.json.gz’ saved [538673481/538673481]



In [7]:
# load the data
data = []
with gzip.open('Kindle_Store_5.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))
# total length of list, this number equals total number of products
print(len(data))
# first row of the list
print(data[0])

2222983
{'overall': 4.0, 'verified': True, 'reviewTime': '07 3, 2014', 'reviewerID': 'A2LSKD2H9U8N0J', 'asin': 'B000FA5KK0', 'style': {'Format:': ' Kindle Edition'}, 'reviewerName': 'sandra sue marsolek', 'reviewText': 'pretty good story, a little exaggerated, but I liked it pretty well.  liked the characters, the plot..it had mystery, action, love, all of the main things. I think most western lovers would injoy this book', 'summary': 'pretty good story', 'unixReviewTime': 1404345600}


In [8]:
# convert list into pandas dataframe
df = pd.DataFrame.from_dict(data)
print(len(df))

2222983


In [9]:
# create a new df with only necessary columns
new_df = df[['overall','reviewText']].copy()
new_df.head()

,overall,reviewText
0,4.0,"pretty good story, a little exaggerated, but I..."
1,5.0,"If you've read other max brand westerns, you k..."
2,5.0,"Love Max, always a fun twist"
3,5.0,"As usual for him, a good book"
4,5.0,MB is one of the original western writers and ...


In [10]:
# split df into positive and negative reviews based on the overall star rating. 
#omitting 3 stars as they could be either + or -
pos_reviews = new_df.loc[new_df['overall'] > 3]
neg_reviews = new_df.loc[new_df['overall'] < 3]

In [11]:
# add column to label review type and combine df back to one 
pos_reviews.insert(2, 'reaction', 'positive')
neg_reviews.insert(2, 'reaction', 'negative')

all_reviews = pd.concat([pos_reviews, neg_reviews], ignore_index=True)
all_reviews.head()

,overall,reviewText,reaction
0,4.0,"pretty good story, a little exaggerated, but I...",positive
1,5.0,"If you've read other max brand westerns, you k...",positive
2,5.0,"Love Max, always a fun twist",positive
3,5.0,"As usual for him, a good book",positive
4,5.0,MB is one of the original western writers and ...,positive


In [12]:
# remove reviews that do not contain text
all_reviews['reviewText'].isnull().sum()
reviews = all_reviews[all_reviews['reviewText'].notna()]
reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2024661 entries, 0 to 2025033
Data columns (total 3 columns):
 #   Column      Dtype  
---  ------      -----  
 0   overall     float64
 1   reviewText  object 
 2   reaction    object 
dtypes: float64(1), object(2)
memory usage: 61.8+ MB


In [13]:
# create a sample df with equal number of + and - reviews 
sample_df = reviews.groupby('reaction').apply(lambda x: x.sample(n=10000)).reset_index(drop = True)
sample_df['reaction'].value_counts()

negative    10000
positive    10000
Name: reaction, dtype: int64

In [14]:
# create lists of + and - reviews from the sample df
pos_df = sample_df.loc[sample_df['reaction'] == 'positive']
pos_list = pos_df['reviewText'].tolist()

neg_df = sample_df.loc[sample_df['reaction'] == 'negative']
neg_list = neg_df['reviewText'].tolist()

In [15]:
# clean the lists (make all lowercase, convert to strings, and remove punctuation)
pos_list_lowered = [word.lower() for word in pos_list]
neg_list_lowered = [word.lower() for word in neg_list]

pos_list_to_string = ' '.join([str(elem) for elem in pos_list_lowered]) 
neg_list_to_string = ' '.join([str(elem) for elem in neg_list_lowered])

nltk.download('stopwords')

stop = set(stopwords.words('english') + list(string.punctuation))

tokenizer = WhitespaceTokenizer()

filtered_pos_list1 = [w for w in tokenizer.tokenize(pos_list_to_string) if w not in stop]
filtered_pos_list = [w.strip(string.punctuation) for w in filtered_pos_list1]

filtered_neg_list1 = [w for w in tokenizer.tokenize(neg_list_to_string) if w not in stop]
filtered_neg_list = [w.strip(string.punctuation) for w in filtered_neg_list1]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [16]:
# view the frequency distribution of the most common + and - words
freq_dist_pos = nltk.FreqDist(filtered_pos_list)
freq_dist_neg = nltk.FreqDist(filtered_neg_list)

print(freq_dist_pos.most_common(15))
print(freq_dist_neg.most_common(15))

[('book', 10650), ('story', 6351), ('read', 6051), ('love', 4798), ('one', 4074), ('series', 3762), ('characters', 2850), ('great', 2790), ('good', 2756), ('like', 2605), ('really', 2578), ('books', 2299), ('loved', 2297), ('well', 2141), ('reading', 2089)]
[('book', 10976), ('story', 6168), ('like', 4669), ('read', 4611), ('one', 4128), ('would', 3127), ('really', 3069), ('author', 2657), ('characters', 2622), ('get', 2569), ('it', 2449), ('good', 2441), ('even', 2258), ('time', 2248), ('books', 2227)]


In [17]:
# convert the sets of words to feature sets
def word_features(words):
    return dict([(word, True) for word in words.split()])

In [18]:
# combine into one set for training and testing
positive_features = [(word_features(f), 'pos') for f in filtered_pos_list]
negative_features = [(word_features(f), 'neg') for f in filtered_neg_list]
labeledwords = positive_features + negative_features

In [19]:
# shuffle the words and train and test the model
random.shuffle(labeledwords)
train_set, test_set = labeledwords[2000:], labeledwords[:500]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [21]:
# test the model w examples
print(classifier.classify(word_features('I loved this book, the story was great')))
print(classifier.classify(word_features('I did not like this book, the characters were bland')))

pos
neg


In [22]:
# calculate the accuracy of the model
print(nltk.classify.accuracy(classifier, test_set))

0.616


In [37]:
# show the model's most informative features from the set
classifier.show_most_informative_features(15)

Most Informative Features
                 deleted = True              neg : pos    =     69.7 : 1.0
                 stilted = True              neg : pos    =     47.1 : 1.0
                  wasted = True              neg : pos    =     45.6 : 1.0
             wonderfully = True              pos : neg    =     40.4 : 1.0
                   waste = True              neg : pos    =     39.8 : 1.0
                    lame = True              neg : pos    =     39.5 : 1.0
                  poorly = True              neg : pos    =     36.8 : 1.0
         inconsistencies = True              neg : pos    =     30.6 : 1.0
                 hotness = True              pos : neg    =     26.8 : 1.0
                     2.5 = True              neg : pos    =     25.8 : 1.0
            heartwarming = True              pos : neg    =     25.6 : 1.0
                    neve = True              pos : neg    =     25.6 : 1.0
                   trite = True              neg : pos    =     25.1 : 1.0

# Classify generated product reviews

In [35]:
path_to_file = '/content/drive/MyDrive/Colab Notebooks/edited_words.txt'
example = open(path_to_file, 'rb').read().decode(encoding='utf-8')
example = example.replace('\n', ' ')
ex_str = str(example)
ex_arr = ex_str.split("\"")

In [36]:
n = 1
for i in ex_arr:
  print("Review ", n, ":", classifier.classify(word_features(i)))
  n = n+1

Review  1 : pos
Review  2 : pos
Review  3 : neg
Review  4 : neg
Review  5 : neg
Review  6 : pos
Review  7 : neg
Review  8 : neg
Review  9 : neg
Review  10 : pos
